# Imports

In [30]:
#Packages
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from statistics import mean
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from keras import Model, regularizers
from keras.layers import Input, Dense, Activation, Dropout, BatchNormalization, Add, Embedding, Bidirectional, LSTM
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve, average_precision_score

from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Data loading

## Mount Drive

In [31]:
drive.mount('/content/drive', force_remount=True)
root_dir = "/content/drive/My Drive/"

Mounted at /content/drive


## Load Dataset 

In [32]:
# Load hotspots kmer list
hotspots = np.load(root_dir+"Data/hotspots/kmers/hotspots-3k-list-500chunk.npy")


# Load labels
labels = np.load(root_dir+"Data/hotspots/kmers/labels_hotspots-3k-list-500chunk.npy")

In [33]:
#[OPTIONAL] limit number of samples to speed up training
hotspots, labels = shuffle(hotspots, labels, random_state = 0)
hotspots = hotspots[0:round((len(hotspots))/1)]
labels = labels[0:round((len(labels))/1)]


In [34]:
print('Hotspots loaded, shape:', hotspots.shape)
print('Labels loaded, shape: ', labels.shape)

Hotspots loaded, shape: (77168, 500)
Labels loaded, shape:  (77168,)


## Load DNA2Vec

In [35]:
!git clone https://github.com/Sergiodiaz53/dna2vec.git
!pip install -r dna2vec/requirements.txt

fatal: destination path 'dna2vec' already exists and is not an empty directory.


In [36]:
from dna2vec.dna2vec.multi_k_model import MultiKModel

In [37]:
K = 3

filepath = 'dna2vec/pretrained/dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v'
mk_model = MultiKModel(filepath)
mk_model = mk_model.model(K)

pretrained_weights = mk_model.vectors
vocab_size, embedding_dim = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

def word2idx(word):
    return mk_model.key_to_index[word]
def idx2word(idx):
  return mk_model.wv.index_to_key[idx]


Result embedding shape: (64, 100)


# Preprocessing data

In [38]:
#######################################################################
#SeqTokenizer##########################################################
#######################################################################

hotspots_sequences = []

for idx, sample in enumerate(hotspots):
    current_seq = []
    for idx2, token in enumerate(sample):
      token = token.upper()
      try:
          model_token = word2idx(token)
          current_seq.append(model_token)
      except:
          current_seq.append("0")

    hotspots_sequences.append(current_seq)

hotspots = hotspots_sequences
seq_size = len(hotspots[0])

# Neural Network

## Hyperparameters

In [39]:
EPOCHS = 50
LEARNING_RATE = 0.01
BATCH_SIZE = 128
DROPOUT_RATE = 0.3

# LSTM
HIDDEN_UNITS_LSTM = 8
HIDDEN_UNITS_DENSE_LSTM = 8
RECURRENT_DROPOUT_RATE = 0
L2_RATE = 1e-05


seq_size = len(hotspots[0])
hotspots = np.array(hotspots)

reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=25, min_delta=0.01, cooldown=25, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=60, restore_best_weights=True)

## Model Definition

In [40]:
def createOptimizer(model, learning_rate):

  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

  model.compile(loss="binary_crossentropy",
                optimizer=optimizer,
                metrics = ['accuracy'])
  return model

def createBidirectionalLSTMModel(seq_lenght, vocab_size, embedding_dim, pretrained_weights_for_embedding):
    initializer = keras.initializers.GlorotNormal()

    model_input = Input(shape=(seq_lenght))
    output = Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        input_length=seq_lenght,
                        weights=[pretrained_weights_for_embedding])(model_input)
    
    output  = Bidirectional(LSTM(units=HIDDEN_UNITS_LSTM, kernel_initializer=initializer,
                                     kernel_regularizer=regularizers.l2(L2_RATE),
                                    dropout=DROPOUT_RATE, recurrent_dropout=RECURRENT_DROPOUT_RATE))(output)    
    output = Dense(1, activation='sigmoid')(output)

    model = Model(inputs=model_input, outputs=output)

    return model



## Training

In [ ]:
test_acc_max = 0
best_model = ""
x_test_max = ""
y_true_max = ""
best_history = ""
scores = []

for i in range(0,1):

  hs_train, hs_test, y_train, y_test = train_test_split(hotspots, labels, test_size=0.2, shuffle=True)

  hs_train = hs_train.astype('float32')
  hs_test = hs_test.astype('float32')

  model = createBidirectionalLSTMModel(seq_size, vocab_size, embedding_dim, pretrained_weights)
  model = createOptimizer(model, LEARNING_RATE)
  history = model.fit(hs_train, y_train, validation_data=(hs_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, verbose=2, callbacks=[reduce_lr, early_stopping])
  test_loss, test_acc = model.evaluate(hs_test, y_test)
  scores.append(test_acc)
  if(test_acc > test_acc_max):
    test_acc_max = test_acc
    best_model = model
    x_test_max = hs_test
    y_true_max = y_test
    best_history = history

print('Max accuracy:', test_acc_max)
print('Mean accuracy:', mean(scores))

Epoch 1/50
483/483 - 105s - loss: 0.6859 - accuracy: 0.5563 - val_loss: 0.6763 - val_accuracy: 0.5816
Epoch 2/50
483/483 - 101s - loss: 0.6760 - accuracy: 0.5782 - val_loss: 0.6739 - val_accuracy: 0.5790
Epoch 3/50
483/483 - 102s - loss: 0.6807 - accuracy: 0.5674 - val_loss: 0.6802 - val_accuracy: 0.5714
Epoch 4/50
483/483 - 102s - loss: 0.6711 - accuracy: 0.5822 - val_loss: 0.6719 - val_accuracy: 0.5897
Epoch 5/50
483/483 - 102s - loss: 0.6683 - accuracy: 0.5956 - val_loss: 0.6688 - val_accuracy: 0.5969
Epoch 6/50
483/483 - 102s - loss: 0.6670 - accuracy: 0.5982 - val_loss: 0.6684 - val_accuracy: 0.5946
Epoch 7/50
483/483 - 102s - loss: 0.6652 - accuracy: 0.6002 - val_loss: 0.6689 - val_accuracy: 0.5899
Epoch 8/50
483/483 - 102s - loss: 0.6647 - accuracy: 0.6019 - val_loss: 0.6784 - val_accuracy: 0.5684
Epoch 9/50
483/483 - 102s - loss: 0.6711 - accuracy: 0.5805 - val_loss: 0.6688 - val_accuracy: 0.5954
Epoch 10/50
483/483 - 102s - loss: 0.6673 - accuracy: 0.5999 - val_loss: 0.6663 - 

# Results


## Report

In [ ]:
y_pred=best_model.predict(x_test_max).ravel()
print(classification_report(y_true_max, (y_pred > 0.5)))

## ROC Curve

In [ ]:
# calling the roc_curve, extract the probability of 
# the positive class from the predicted probability
fpr, tpr, thresholds = roc_curve(y_true_max, y_pred)

# AUC score that summarizes the ROC curve
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, lw = 2, label = 'ROC AUC: {:.2f}'.format(roc_auc))
plt.plot([0, 1], [0, 1],
         linestyle = '--',
         color = (0.6, 0.6, 0.6),
         label = 'random guessing')
plt.plot([0, 0, 1], [0, 1, 1],
         linestyle = ':',
         color = 'black', 
         label = 'perfect performance')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.title('Receiver Operator Characteristic')
plt.legend(loc = "lower right")
plt.tight_layout()
plt.show()

## Precission Recall Curve

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_true_max, y_pred)

# AUC score that summarizes the precision recall curve
avg_precision = average_precision_score(y_true_max, y_pred)

label = 'Precision Recall AUC: {:.2f}'.format(avg_precision)
plt.plot(recall, precision, lw = 2, label = label)
plt.xlabel('Recall')  
plt.ylabel('Precision')  
plt.title('Precision Recall Curve')
plt.legend()
plt.tight_layout()
plt.show()

## Confussion Matrix

In [ ]:
y_pred=best_model.predict(x_test_max).ravel()
y_pred = y_pred > 0.5

class_names = ["Hotspot", "No Hotspot"]
con_mat = tf.math.confusion_matrix(labels=y_true_max, predictions=y_pred).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
con_mat_df = pd.DataFrame(con_mat_norm, index = class_names, columns = class_names)

print('Accuracy Y_test: ', accuracy_score(y_true_max, y_pred))
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

## Accuracy

In [ ]:
plt.plot(best_history.history['accuracy'])
plt.plot(best_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Loss

In [ ]:
plt.plot(best_history.history['loss'])
plt.plot(best_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Save Data


In [ ]:
model.save(root_dir+'ResNetmodel-2kEpochs.h5')